In [26]:
import pandas as pd
from datetime import timedelta
import numpy as np

# Read datasets in
> Ensure that zipcdes are read as strings and not numeric

In [27]:
# Load the customer dataset
file_path = "data/Customer_sales.csv"
df_customer_sales = pd.read_csv(file_path,dtype={"BillPostalCode":"string"} )
df_customer_sales

,CustomerID,BillPostalCode,BillStateProvCode,BillCountryCode,LOB,Market,ChannelOrderID,OrderDate,Quantity,UnitPrice,Total,SKU,Title,Brand
0,10012,49649,MI,US,Salvage,eBay,4468548,2021-04-29,1,25.00,25.00,883562,Can-Am Commander 1000 XT 14 Fender Left Front ...,Can-Am
1,10012,49649,MI,US,Salvage,eBay,4468548,2021-04-29,1,45.00,45.00,983570,Can-Am Commander 800R XT 12 Fender Right Front...,Can-Am
2,10012,49649,MI,US,Salvage,eBay,4468639,2021-04-29,1,31.99,31.99,1219950,Can-Am Commander 1000 XT 14 Fender Flare Left ...,Can-Am
3,10012,49649,MI,US,Salvage,eBay,4471261,2021-04-30,1,19.99,19.99,1054150,Can-Am Commander 1000 XT 11 Lateral Panel Righ...,Can-Am
4,24012,68713,NE,US,Salvage,Website,4758999,2021-09-07,1,69.99,69.99,1302547,Polaris Sportsman 90 02 Footrest Left 0450616 ...,Polaris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237096,374531012,70578,LA,US,Salvage,Website,5616208,2023-01-20,1,19.95,19.95,1651845,Honda Rubicon 500 FPA 13 A Arm Left Front Uppe...,Honda
237097,374532012,54758,WI,US,Salvage,Website,5616192,2023-01-20,1,35.92,35.92,1594302,Arctic Cat Prowler 700 HDX 12 Hub Front Left o...,Arctic Cat
237098,374533012,77868,TX,US,Salvage,Website,5616193,2023-01-20,1,807.45,807.45,1541219,Can-Am Outlander 500 DPS 13 Transmission 42068...,Can-Am
237099,374534012,47933,IN,US,Salvage,eBay,5616154,2023-01-20,1,54.98,54.98,PSN-5439014-070,Polaris Ranger 1000 900 570 Diesel Upper Dash...,Polaris


In [28]:
# Group by order and customer - total sales
Orders_df = df_customer_sales.groupby(['ChannelOrderID','OrderDate','CustomerID']).agg({'Total': lambda x:x.sum()}).reset_index()
Orders_df

,ChannelOrderID,OrderDate,CustomerID,Total
0,4196075,2020-12-31,240414012,39.99
1,4196093,2020-12-31,240413012,250.00
2,4196116,2020-12-31,240411012,19.99
3,4196117,2020-12-31,229366012,49.98
4,4196129,2020-12-31,240475012,129.99
...,...,...,...,...
204292,5616290,2023-01-20,374524012,49.99
204293,5616291,2023-01-20,374522012,187.24
204294,5616300,2023-01-20,374521012,38.99
204295,5616301,2023-01-20,374520012,18.99


In [29]:
# Convert orderdate to datetime type so calculations can be done
Orders_df['OrderDate'] = Orders_df['OrderDate'].astype('datetime64[ns]')

In [30]:
Orders_df.dtypes

ChannelOrderID             int64
OrderDate         datetime64[ns]
CustomerID                 int64
Total                    float64
dtype: object

In [31]:
# Set variables for NOW and and period
NOW = Orders_df['OrderDate'].max() + timedelta(days=1)
NOW

Timestamp('2023-01-21 00:00:00')

In [32]:
# Data is for a two year period
period = 730

In [33]:
# Add column for Days since order to aid with calcuation
Orders_df['DaysSinceOrder'] = Orders_df['OrderDate'].apply(lambda  x: (NOW -x).days)

In [34]:
aggr = {
    'DaysSinceOrder' : lambda x : x.min(),
    'OrderDate' : lambda x: len([d for d in x if d>=NOW - timedelta(days=period)])
}

rfm_df = Orders_df.groupby('CustomerID').agg(aggr).reset_index()
rfm_df.rename(columns={'DaysSinceOrder' : 'Recency', 'OrderDate' : 'Frequency'}, inplace=True)
rfm_df

,CustomerID,Recency,Frequency
0,10012,631,3
1,24012,45,11
2,36012,43,7
3,48012,10,42
4,49012,38,14
...,...,...,...
140781,374531012,1,1
140782,374532012,1,1
140783,374533012,1,1
140784,374534012,1,1


In [35]:
# rfm_df['Monetary'] = rfm_df['CustomerID'].apply(lambda x: Orders_df[('CustomerID'] == x) & Orders_df
# ['OrderDate'] >= NOW - timedelta(days=period)['Total'].sum())




rfm_df['Monetary'] = rfm_df['CustomerID'].apply(lambda x: Orders_df[(Orders_df['CustomerID'] == x) &
                                                           (Orders_df['OrderDate'] >= NOW - timedelta(days=period))]
                                                           ['Total'].sum())

rfm_df

,CustomerID,Recency,Frequency,Monetary
0,10012,631,3,121.98
1,24012,45,11,1152.57
2,36012,43,7,268.83
3,48012,10,42,7014.86
4,49012,38,14,2460.47
...,...,...,...,...
140781,374531012,1,1,43.94
140782,374532012,1,1,35.92
140783,374533012,1,1,807.45
140784,374534012,1,1,54.98


In [36]:
quintiles = rfm_df[['Recency', 'Frequency', 'Monetary']].quantile([.2, .4, .6, .8]).to_dict()
quintiles

{'Recency': {0.2: 141.0, 0.4: 282.0, 0.6: 445.0, 0.8: 594.0},
 'Frequency': {0.2: 1.0, 0.4: 1.0, 0.6: 1.0, 0.8: 1.0},
 'Monetary': {0.2: 18.99, 0.4: 37.98, 0.6: 72.99, 0.8: 174.97}}

In [37]:
def r_score(x):
    if x <= quintiles['Recency'][.2]:
        return 5
    elif x <= quintiles['Recency'][.4]:
        return 4
    elif x <= quintiles['Recency'][.6]:
        return 3
    elif x <= quintiles['Recency'][.8]:
        return 2
    else:
        return 1

def fm_score(x, c):
    if x <= quintiles[c][.2]:
        return 1
    elif x <= quintiles[c][.4]:
        return 2
    elif x <= quintiles[c][.6]:
        return 3
    elif x <= quintiles[c][.8]:
        return 4
    else:
        return 5

In [38]:
rfm_df['R'] = rfm_df['Recency'].apply(lambda x: r_score(x))
rfm_df['F'] = rfm_df['Frequency'].apply(lambda x: fm_score(x, 'Frequency'))
rfm_df['M'] = rfm_df['Monetary'].apply(lambda x: fm_score(x, 'Monetary'))

rfm_df

,CustomerID,Recency,Frequency,Monetary,R,F,M
0,10012,631,3,121.98,1,5,4
1,24012,45,11,1152.57,5,5,5
2,36012,43,7,268.83,5,5,5
3,48012,10,42,7014.86,5,5,5
4,49012,38,14,2460.47,5,5,5
...,...,...,...,...,...,...,...
140781,374531012,1,1,43.94,5,1,3
140782,374532012,1,1,35.92,5,1,2
140783,374533012,1,1,807.45,5,1,5
140784,374534012,1,1,54.98,5,1,3


In [39]:
rfm_df['RFM Score'] = rfm_df['R'].map(str) + rfm_df['F'].map(str) + rfm_df['M'].map(str)
rfm_df.head()

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM Score
0,10012,631,3,121.98,1,5,4,154
1,24012,45,11,1152.57,5,5,5,555
2,36012,43,7,268.83,5,5,5,555
3,48012,10,42,7014.86,5,5,5,555
4,49012,38,14,2460.47,5,5,5,555


In [40]:
segt_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at risk',
    r'[1-2]5': 'can\'t loose',
    r'3[1-2]': 'about to sleep',
    r'33': 'need attention',
    r'[3-4][4-5]': 'loyal customers',
    r'41': 'promising',
    r'51': 'new customers',
    r'[4-5][2-3]': 'potential loyalists',
    r'5[4-5]': 'champions'
}

rfm_df['Segment'] = rfm_df['R'].map(str) + rfm_df['F'].map(str)
rfm_df['Segment'] = rfm_df['Segment'].replace(segt_map, regex=True)
rfm_df

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM Score,Segment
0,10012,631,3,121.98,1,5,4,154,can't loose
1,24012,45,11,1152.57,5,5,5,555,champions
2,36012,43,7,268.83,5,5,5,555,champions
3,48012,10,42,7014.86,5,5,5,555,champions
4,49012,38,14,2460.47,5,5,5,555,champions
...,...,...,...,...,...,...,...,...,...
140781,374531012,1,1,43.94,5,1,3,513,new customers
140782,374532012,1,1,35.92,5,1,2,512,new customers
140783,374533012,1,1,807.45,5,1,5,515,new customers
140784,374534012,1,1,54.98,5,1,3,513,new customers


In [41]:
rfm_df.to_csv('data/rfm_data.csv')